In [27]:
# coding: utf-8
import sys
import ast
from ast import AST
import tokenize
import token
from numbers import Number
import json
import io

In [61]:
def tokenASTCombine(tok, astNode):
    tok.pop(0)
    nodeyList, looseTokenList, tk = zipTokAST(tok, astNode, []) #skip start marker token
    while(len(tk) > 1): #skip the end maker 
        if(tk[0].type != token.NEWLINE):
            nodeyList.append(formatToken(tk[0]))
        tk.pop(0)
    return json.dumps(nodeyList)


def formatToken(tk):
    return {'type': token.tok_name[tk.type], 'start': {'line': tk.start[0], 'col': tk.start[1]}, 'end': {'line': tk.end[0], 'col': tk.end[1]}, 'literal': tk.string}


def zipTokAST(tk, node, nodeyList):
    if(len(tk) < 1 or tk[0].type == token.ENDMARKER):
        return nodeyList, [], [] #end of tokenszm
    elif(node == None):
        return nodeyList, [], [] #end of nodes
    
    childrenZip = []
    for child in ast.iter_child_nodes(node): #depth fist
        cz, tz, tk = zipTokAST(tk, child, [])
        if(len(tz) > 0):
            childrenZip += tz
        if(len(cz) > 0):
            childrenZip += cz
 
    list.sort(childrenZip, key = lambda x : x['start']['line']*10000+x['start']['col'] )
        
    marker = {'type': type(node).__name__, 'content': childrenZip}
    looseTokenList = []
    if(len(childrenZip) > 0):
        marker['start'] = childrenZip[0]['start']
        marker['end'] = childrenZip[len(childrenZip) - 1]['end']
    
    if(hasattr(node, 'lineno')): #meaning it's a node that appears in the text
        line = node.lineno
        col = node.col_offset
        #print("node is", type(node).__name__, line, col, tk[0])
        # check for nodes that don't belong to anyone
        while(len(tk) > 0 and (tk[0].start[0] < line or (tk[0].start[0] == line and tk[0].start[1] < col))): #actually starts before this node
            if(tk[0].type != token.NEWLINE):
                looseTokenList.append(formatToken(tk[0]))
            tk.pop(0)
        if(len(tk) > 0 and tk[0].start[0] == line and tk[0].start[1] == col):
            if(tk[0].type != token.NEWLINE):
                formatted = formatToken(tk[0])
                marker['start'] = formatted['start']
                marker['end'] = formatted['end']
                marker['literal'] = formatted['literal']
            tk.pop(0)
            
    if('literal' in marker or len(childrenZip) > 0):
        nodeyList.append(marker)
  
    return nodeyList, looseTokenList, tk
        
                

In [62]:
def makeNodey(node):
    jsn = {'type': type(node).__name__, 'node_uid': 0, 'content': []}

    if(hasattr(node, 'lineno')): #meaning it's a node that appears in the text
        jsn['line'] = node.lineno
        jsn['col'] = node.col_offset

    for child in ast.iter_child_nodes(node):
        c = makeNodey(child)
        if(c and jsn):
            jsn['content'].append(c)

    for field, value in ast.iter_fields(node):
        #print("fiel?", field, value)
        if(isinstance(value, str) or isinstance(value, Number) or isinstance(value, bool)):
            #print("fiel is", field, value)
            value = json.dumps(value)
            if(len(jsn['content']) > 0): #add as a name Nodey
                lit = {'type': 'Name', 'literal': value, 'node_uid': 0, 'content': [], 'line': node.lineno, 'col': node.col_offset}
                jsn['content'].append(lit)
            else:
                jsn['literal'] = value

    if(len(jsn['content']) > 0 or 'literal' in jsn):
        return jsn

    

In [63]:
def parseCode(str):
    #print("got code", str)
    code = str.encode('utf-8')
    tree = ast.parse(str)
    bytes = io.BytesIO(code)
    g = tokenize.tokenize(bytes.readline)
    tk = list(g)
    print(tokenASTCombine(tk, tree))

In [64]:
text = "formatToken({'foo': 7})\njson.dumps(jsn)"
parseCode(text)

trying to sort [] 

trying to sort [] 

trying to sort [] 

trying to sort [] 

trying to sort [{'type': 'OP', 'start': {'line': 1, 'col': 11}, 'end': {'line': 1, 'col': 12}, 'literal': '('}, {'type': 'OP', 'start': {'line': 1, 'col': 12}, 'end': {'line': 1, 'col': 13}, 'literal': '{'}, {'type': 'Str', 'content': [], 'start': {'line': 1, 'col': 13}, 'end': {'line': 1, 'col': 18}, 'literal': "'foo'"}, {'type': 'OP', 'start': {'line': 1, 'col': 18}, 'end': {'line': 1, 'col': 19}, 'literal': ':'}, {'type': 'Num', 'content': [], 'start': {'line': 1, 'col': 20}, 'end': {'line': 1, 'col': 21}, 'literal': '7'}] 

trying to sort [{'type': 'Name', 'content': [], 'start': {'line': 1, 'col': 0}, 'end': {'line': 1, 'col': 11}, 'literal': 'formatToken'}, {'type': 'Dict', 'content': [{'type': 'OP', 'start': {'line': 1, 'col': 11}, 'end': {'line': 1, 'col': 12}, 'literal': '('}, {'type': 'OP', 'start': {'line': 1, 'col': 12}, 'end': {'line': 1, 'col': 13}, 'literal': '{'}, {'type': 'Str', 'content': 